# Directory

> directory class.

In [ ]:
#| default_exp utils.directory

In [ ]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
import os, pathlib 
from pathlib import Path
from dataclasses import dataclass, field, KW_ONLY
from typing import Optional, List, ClassVar, Any, TypeAlias, Callable
from enum import StrEnum

In [2]:
#| export
from iza.types import (
    PathLike, PathType,
    DirectoryTreeStrings, TreeEntryFunc,
    RichTree, RichText, RichConsole, RichProgress
)
from iza.static import EXT_PY

#| export
from iza.imp import RichImp
from ipos.imp import is_mod, is_var_imp, Module, Imp

In [ ]:
#| export
try:    
    from rich.tree import Tree
    from rich.text import Text    
    from rich.filesize import decimal
    from rich import get_console
    from rich.console import Console
    from rich.progress import Progress

except ImportError:
    Tree = None
    Text = None 
    Progress = None

### Directory Viewer

#### Default Functions

##### Init Tree

In [ ]:
#| export
def base_init_tree(dirname: str):
    return None

def rich_init_tree(dirname):
    return Tree(f'[link file://{dirname}]{dirname}', guide_style='bold bright_blue')

def init_tree(dirname):
    try:
        return rich_init_tree(dirname)
    except:
        return base_init_tree(dirname)

##### Walked Entry to Str

In [ ]:
#| export
def base_entry_fn(path:Path, prefix:str='', pointer:str='', suffix:str='') -> str:
    name = path.name
    return f'{prefix}{pointer}{name}{suffix}'

def rich_entry_fn(path:Path, prefix:str='', pointer:str='', suffix:str='') -> str:
    text = Text(path.name)
    size = decimal(path.stat().st_size)
    text.stylize(f'link file://{path}')
    text.append(f' ({size})', 'cyan')
    return text

def entry_fn(path:Path, prefix:str='', pointer:str='', suffix:str='') -> str:
    try:
        return rich_entry_fn(path, prefix, pointer, suffix)
    except:
        return base_entry_fn(path, prefix, pointer, suffix)


#### Walk Function

In [ ]:
#| export
def walk_dir_tree(
    dirname: PathLike, 
    prefix: str = '',
    hidden: Optional[bool] = False, 
    tree: Optional[RichTree] = None,
    entry_fn: TreeEntryFunc = entry_fn,
):    
    '''
    A recursive generator, given a directory Path object
    will yield a visual tree structure line by line
    with each line prefixed by the same characters
    Notes
    -----
    Adapted from https://stackoverflow.com/a/59109706/5623899
    '''
    # NOTE: sort_directory_first, to_abs_expanded, rich_file defined in _02_utils/_08_modules.ipynb
    dirname = Path(to_abs_expanded(dirname))
    contents = sorted(Path(dirname).iterdir(), key=sort_directory_first)
    
    SPACE, BRANCH, TEE, LAST = DirectoryTreeStrings
    pointers = [TEE] * (len(contents) - 1) + [LAST]
    
    for pointer, path in zip(pointers, contents):
        # Remove hidden files
        if path.name.startswith('.') and not hidden:
            continue
        
        name = entry_fn(path, prefix, pointer, suffix='')
        yield name
        
        branch = None
        if tree is not None:
            branch = tree.add(entry_fn(path))
    
        if path.is_dir():
            # NOTE: space because last, └── , above so no more |
            extension = BRANCH if pointer == TEE else SPACE
            yield from walk_dir_tree(path, prefix=f'{prefix}{extension}', hidden=hidden, tree=branch, entry_fn=entry_fn)

#### DirectoryTree
walks through directory

In [128]:
#| export
@dataclass
class DirectoryTree:
    dirname: str
    
    hidden: Optional[bool] = False
    entry_fn: Optional[TreeEntryFunc] = field(default=entry_fn)
    tree: Optional[RichTree] = field(init=False, default=None)
    
    def __post_init__(self):
        # NOTE: defined in _02_utils/_01_files.ipynb
        self.dirname = Path(to_abs_expanded(self.dirname))
        if self.tree is None:
            self.tree = init_tree(self.dirname)

    def tree_generator(self):
        yield from walk_dir_tree(
            self.dirname, hidden=self.hidden, 
            tree=self.tree, entry_fn=self.entry_fn
        )
        
    def get_tree_lines(self) -> List[str]:        
        tree_gen = self.tree_generator()
        lines = [line for line in tree_gen]
        return lines
    
    def make_tree_str(self) -> str:        
        lines = self.get_tree_lines()
        tree_str = '\n'.join([str(self.dirname), *lines])
        return tree_str
   
    def print(self) -> None:        
        tree_str = self.make_tree_str()        
        print(tree_str)
        return

    def __repr__(self):        
        tree_str = self.make_tree_str()        
        return tree_str  

In [ ]:
#| export 
@dataclass
class RichDirectory(DirectoryTree):
    _: KW_ONLY
    console: Optional[RichConsole] = field(default_factory=Console, init=True, repr=False)    
    _imp: Imp = field(default_factory=RichImp, init=False, repr=False)
    def __post_init__(self):
        super().__post_init__()
        self._imp = RichImp()
    
        if self.console is None and is_mod(self._imp._module):
            self.console = get_console()

    def print_rich(self) -> None:    
        lines = self.get_tree_lines()
        self.console.print(self.tree)

In [ ]:
#| export 
@dataclass
class Directory(DirectoryTree):
    dirname: PathLike
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()